## libraries

In [1]:
#!pip install nltk
#!pip install textdistance
import pandas as pd
import regex
import string
import nltk
from nltk.corpus import stopwords
import textdistance as td
import numpy as np

## cleaned dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/satyam9090/Comparing-Cosmetics-by-Ingredients/master/datasets/cosmetics.csv')
df.head()
starts_with_visit = df[df.Ingredients.str.startswith("Visit")]
df = pd.concat([df, starts_with_visit]).drop_duplicates(keep=False) #we are droping the duplicate values, the ones whose ingredients start with Visit are dropped
df.Ingredients.str.startswith("Visit").sum() #checking whether there are any that start with Visit
df = df[~df.Ingredients.isin(["#NAME?","No Info"])]
starts_with_st_change = df[df.Ingredients.str.startswith("This ingredient list is subject to change")]
df = pd.concat([df,starts_with_st_change]).drop_duplicates(keep = False)
starts_with_asteriks = df[df.Ingredients.str.startswith("*")]
df = pd.concat([df, starts_with_asteriks]).drop_duplicates(keep = False)
df = df[df.Ingredients != '-4MSK: Helps remove melanin trapped in dark spots and ion force complex works to slough off the surface cells that contain melanin.']

## test variables

## cleaning the string

In [122]:
nltk.download('stopwords')
stopwords_ = stopwords.words('english')
def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = ' '.join([word for word in text.split() if word not in stopwords_])
    return text
cleaned = clean_string(ingredients)
print(cleaned)

Water Glycolic Acid Butylene Glycol Glycerin Sodium Hydroxide Salicylic Acid Lactic Acid Citric Acid Vitis Vinifera Grape Juice Extract Aloe Barbadensis Leaf Juice Opuntia FicusIndica Extract Aesculus Hippocastanum Horse Chestnut Seed Extract Camellia Sinensis Leaf Extract Rubus Idaeus Raspberry Fruit Extract Saccharomyces Cerevisiae Extract Buddleja Davidii Meristem Cell Culture Sclerocarya Birrea Seed Oil Sodium Hyaluronate Crosspolymer Allantoin Hydroxyethylcellulose Galactoarabinan Propanediol Disodium EDTA Xanthan Gum Hexylene Glycol Phenoxyethanol Caprylyl Glycol Ethylhexylglycerin Sodium Nitrate Potassium Sorbate Pentylene Glycol Sodium Benzoate


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## function

In [168]:
def cosmetics_with_ingredients(cleaned, label, skintype):
    if ('Not sure' in label) and ('Not sure' in skintype):
        
        df_high_rank = df[df.Rank>=4]
        df_high_rank.reset_index(drop=True, inplace=True)
        matches = []
        indexes = []
        for i in df_high_rank.Ingredients:
            matches.append(td.sorensen.normalized_similarity(cleaned,i))
        for i, value in enumerate(matches):
            if value == max(matches):
                indexes.append(i)
        
        if max(matches)*100>=70:
            skintypes_test = {}
            for i in indexes:
                skins = []
                the_best_one_test = df_high_rank[df_high_rank.index == i]
                if the_best_one_test.Combination[i] == 1:
                    skins.append('Combination')
                if the_best_one_test.Dry[i] == 1:
                    skins.append('Dry')
                if the_best_one_test.Normal[i] == 1:
                    skins.append('Normal')
                if the_best_one_test.Oily[i] == 1:
                    skins.append('Oily')
                if the_best_one_test.Sensitive[i] ==1:
                    skins.append('Sensitive') 
                skintypes_test[i] = skins
            
                
            
            more_skins = []
            for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
                if len(v) >= 4:
                    more_skins.append(i)
                
            
            final = df_high_rank.iloc[more_skins]
            final = final[final.columns[~(final.columns == 'Ingredients')]]
            final.reset_index(drop=True, inplace=True)
            final.sort_values('Rank', ascending=False, inplace=True)
            return final.to_dict('records'), [{"name": i, "id": i} for i in final.columns]
            output_text = ("With the given ingredients, we found the following profucts. This table should help you decide whether you can use the cosmetics or not.")
        else:
            print('Sorry, we are not familiar with such a combination of cosmetics.')
    elif 'Not sure' not in label and 'Not sure' in skintype:
                df_label = df.loc[df.Label.apply(lambda x: x in label)]
                matches = []
                indexes = []
                for i in df_label.Ingredients:
                    matches.append(td.sorensen.normalized_similarity(cleaned,i))
                for i, value in enumerate(matches):
                    if value == max(matches):
                        indexes.append(i)
            
                if max(matches)*100>=70:
                    final = df_label.iloc[indexes, :]
                    display(final)
                else:
                    output_text = 'Sorry, we are not familiar with such a combination of cosmetics.'

    elif ('Not sure' not in label) and ('Not sure' not in skintype):
        df_label = df[df.Label.apply(lambda x: x in label)]
        matches = []
        indexes = []
        for i in df_label.Ingredients:
            matches.append(td.sorensen.normalized_similarity(cleaned,i))
        for i, value in enumerate(matches):
            if value == max(matches):
                indexes.append(i)
                
        if max(matches)*100>=70:
            skintypes_test = {}
            for i in indexes:
                the_best_one_test = df_label.iloc[i, ]
                
                skins = []
                if the_best_one_test.Combination == 1:
                    skins.append('Combination')
                if the_best_one_test.Dry == 1:
                    skins.append('Dry')
                if the_best_one_test.Normal == 1:
                    skins.append('Normal')
                if the_best_one_test.Oily == 1:
                    skins.append('Oily')
                if the_best_one_test.Sensitive ==1:
                    skins.append('Sensitive') 
                skintypes_test[i]=skins
    
            matched=[]
            if len(skins)==0:
                matches=[]
                
            elif len(skins)!=0:
                for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
                    
                    if set(skintype).issubset(set(v)):
                        matched.append(i)
            if len(matches)!=0:
                output_text = 'This cosmetics fits your skin type with '+str(int(max(matches)*100))+'%'
                print(output_text)
            else:
                output_text = 'This cosmetics does not match your skin type, please use our first page to get a recommendation from us',
                print(output_text)
        else:
            output_text = 'Sorry, we are not familiar with such a combination of cosmetics.'
            
    elif ('Not sure' in label) and ('Not sure' not in skintype):
        
        matches = []
        for i in df.Ingredients:
            matches.append(td.sorensen.normalized_similarity(cleaned,i))
        for i, value in enumerate(matches):
            if value == np.max(matches):
                max_index = i
        
        if max(matches)*100>=70:
            
            the_best_cosmetics = df[df.index == max_index]
            skintypes = []
            if the_best_cosmetics.Combination[max_index] == 1:
                skintypes.append('Combination')
            if the_best_cosmetics.Dry[max_index] == 1:
                skintypes.append('Dry')
            if the_best_cosmetics.Normal[max_index] == 1:
                skintypes.append('Normal')
            if the_best_cosmetics.Oily[max_index] == 1:
                skintypes.append('Oily')
            if the_best_cosmetics.Sensitive[max_index] == 1:
                skintypes.append('Sensitive')
                
            
            if set(skintype).issubset(set(skintypes)):
                
                output_text = 'These ingredients are okay for your skin type! You can use the cosmetics with the given ingredients'
                output_text = 'The cosmetics matches with your skin with approximately', int(max(matches)*100), '%'
                print(output_text)
        else:
            output_text = 'Sorry, we are not familiar with such a combination of cosmetics.'
            print(output_text)

In [169]:
#ingredients='Algae (Seaweed) Extract, Mineral Oil, Petrolatum, Glycerin, Isohexadecane, Microcrystalline Wax, Lanolin Alcohol, Citrus Aurantifolia (Lime) Extract, Sesamum Indicum (Sesame) Seed Oil, Eucalyptus Globulus (Eucalyptus) Leaf Oil, Sesamum Indicum (Sesame) Seed Powder, Medicago Sativa (Alfalfa) Seed Powder, Helianthus Annuus (Sunflower) Seedcake, Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal, Sodium Gluconate, Copper Gluconate, Calcium Gluconate, Magnesium Gluconate, Zinc Gluconate, Magnesium Sulfate, Paraffin, Tocopheryl Succinate, Niacin, Water, Beta-Carotene, Decyl Oleate, Aluminum Distearate, Octyldodecanol, Citric Acid, Cyanocobalamin, Magnesium Stearate, Panthenol, Limonene, Geraniol, Linalool, Hydroxycitronellal, Citronellol, Benzyl Salicylate, Citral, Sodium Benzoate, Alcohol Denat., Fragrance.'
ingredients = df.Ingredients[299]
label = ['Cleanser']
skintype = ['Oily']


## testing function


In [171]:
cosmetics_with_ingredients(cleaned, label, skintype)

Label                                                   Cleanser
Brand                                             DRUNK ELEPHANT
Name                       T.L.C. Framboos™ Glycolic Night Serum
Price                                                         90
Rank                                                         4.3
Ingredients    Water, Glycolic Acid, Butylene Glycol, Glyceri...
Combination                                                    1
Dry                                                            1
Normal                                                         1
Oily                                                           1
Sensitive                                                      0
Name: 299, dtype: object

This cosmetics fits your skin type with 97%


## what to improve
- when ingredients are given, bot skin type and label are not.(is not working properly)


In [30]:
df_label = df[df.Label.apply(lambda x: x in label)]
skins = []
the_best_one_test = df_label.iloc[120, ]
if the_best_one_test.Combination ==1:
    skins.append('Combination')
if the_best_one_test.Dry == 1:
    skins.append('Dry')
if the_best_one_test.Normal == 1:
    skins.append('Normal')
if the_best_one_test.Oily == 1:
    skins.append('Oily')
if the_best_one_test.Sensitive ==1:
    skins.append('Sensitive') 
    
the_best_one_test 
skins

['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

In [ ]:
def cosmetics(label, skintype):
    if ('Not sure' not in skintype) and ('Not sure' in label):
        df_full = df[df.Rank >=4.5]
        skintypes_test = {}
        for i in df_full.index:
            skins = []
            the_best_one_test = df_full[df_full.index == i]
            if the_best_one_test.Combination[i] == 1:
                skins.append('Combination')
            if the_best_one_test.Dry[i] == 1:
                skins.append('Dry')
            if the_best_one_test.Normal[i] == 1:
                skins.append('Normal')
            if the_best_one_test.Oily[i] == 1:
                skins.append('Oily')
            if the_best_one_test.Sensitive[i] ==1:
                skins.append('Sensitive') 
            skintypes_test[i] = skins
        matched=[]
        for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
            if set(skintype).issubset(set(v)):
                matched.append(i)
        final = df_full.loc[matched]
        final.drop_duplicates(inplace = True)
        final.sort_values('Rank', ascending=False, inplace=True)
        final.drop_duplicates('Label', inplace=True)
        final = final[['Label','Brand', 'Name', 'Price', 'Rank']]
        final.reset_index(inplace = True, drop=True)
        
        display(final)
    elif ('Not sure' in skintype) and ('Not sure' not in label):
        df_label = df[df.Label.apply(lambda x: x in label)]
        df_label = df_label[df_label.Rank >=4.5]
        skintypes_test = {}
        for i in df_label.index:
            skins = []
            the_best_one_test = df_label[df_label.index == i]
            if the_best_one_test.Combination[i] == 1:
                skins.append('Combination')
            if the_best_one_test.Dry[i] == 1:
                skins.append('Dry')
            if the_best_one_test.Normal[i] == 1:
                skins.append('Normal')
            if the_best_one_test.Oily[i] == 1:
                skins.append('Oily')
            if the_best_one_test.Sensitive[i] ==1:
                skins.append('Sensitive') 
            skintypes_test[i] = skins

        more_skins = []
        for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
            if len(v) == 5:
                more_skins.append(i)
        if len(more_skins)==0:
            for i, v in zip(skintypes_test.keys(), skintypes_test.values()):  
                if len(v) == len(max(skintypes_test.values())):
                    more_skins.append(i)


        final = df_label.loc[more_skins]
        final.drop_duplicates(inplace = True)
        final.sort_values('Rank', ascending=False, inplace=True)
        final.drop_duplicates('Brand', inplace=True)
        final = final[['Brand', 'Name', 'Price', 'Oily', 'Combination', 'Dry', 'Sensitive', 'Normal']]
        final.reset_index(inplace = True, drop=True)
        final = final.style.set_properties(**{'background-color': 'thistle',                                                   
                                      'color': 'navy',                       
                                      'border-color': 'purple4'})
        display(final)

    elif ('Not sure' not in skintype) and ('Note sure' not in label):
        df_label = df[df.Label.apply(lambda x: x in label)]
        df_label = df_label[df_label.Rank >=4.5]
        skintypes_test = {}
        for i in df_label.index:
            skins = []
            the_best_one_test = df_label[df_label.index == i]
            if the_best_one_test.Combination[i] == 1:
                skins.append('Combination')
            if the_best_one_test.Dry[i] == 1:
                skins.append('Dry')
            if the_best_one_test.Normal[i] == 1:
                skins.append('Normal')
            if the_best_one_test.Oily[i] == 1:
                skins.append('Oily')
            if the_best_one_test.Sensitive[i] ==1:
                skins.append('Sensitive') 
            skintypes_test[i] = skins

        matched=[]
        for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
            if set(skintype).issubset(v):
                matched.append(i)

        final = df_label.loc[matched]
        final.drop_duplicates(inplace = True)
        final.sort_values('Rank', ascending=False, inplace=True)
        final.drop_duplicates('Brand', inplace=True)
        final = final[['Brand', 'Name', 'Price']]
        final.reset_index(inplace = True, drop=True)
        final = final.style.set_properties(**{'background-color': 'thistle',                                                   
                                              'color': 'navy',                       
                                              'border-color': 'purple4'})
        display(final)
    elif ('Not sure' in label) and ('Not sure' in skintype):
        df_full = df[df.Rank >=4.5]
        skintypes_test = {}
        for i in df_full.index:
            skins = []
            the_best_one_test = df_full[df_full.index == i]
            if the_best_one_test.Combination[i] == 1:
                skins.append('Combination')
            if the_best_one_test.Dry[i] == 1:
                skins.append('Dry')
            if the_best_one_test.Normal[i] == 1:
                skins.append('Normal')
            if the_best_one_test.Oily[i] == 1:
                skins.append('Oily')
            if the_best_one_test.Sensitive[i] ==1:
                skins.append('Sensitive') 
            skintypes_test[i] = skins

        more_skins = []
        for i, v in zip(skintypes_test.keys(), skintypes_test.values()):
            if len(v) == 5:
                more_skins.append(i)
        if len(more_skins)==0:
            for i, v in zip(skintypes_test.keys(), skintypes_test.values()):  
                if len(v) == len(max(skintypes_test.values())):
                    more_skins.append(i)


        final = df_full.loc[more_skins]
        final.drop_duplicates(inplace = True)
        final.sort_values('Rank', ascending=False, inplace=True)
        final.drop_duplicates('Label', inplace=True)
        final = final[['Label','Brand', 'Name', 'Price', 'Rank','Oily', 'Combination', 'Dry', 'Sensitive', 'Normal']]
        final.reset_index(inplace = True, drop=True)
        
        display(final)

In [ ]:
cosmetics(label, skintype)